> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Lecon 5 : Mon premier LLM

## On assemble tout !

Tu as appris :
1. **Predire la suite** avec des probabilites
2. **Apprendre de ses erreurs** avec la loss et le gradient
3. **Les embeddings** pour donner une memoire au modele
4. **L'attention** pour regarder les lettres importantes

Maintenant, on met tout ensemble pour creer un **vrai mini-LLM**
qui genere des noms de Pokemon inventes !

In [ ]:
# ============================================================
# Cellule d'initialisation \u2014 execute sans lire (Shift+Entree)
# ============================================================

import json
import uuid

from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 3


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_barres(valeurs, etiquettes, titre="Probabilites", couleur="#1565c0"):
    """Barres de probabilites animees (apparition progressive + hover)."""
    max_val = max(valeurs) if valeurs else 1
    n = len(valeurs)
    bar_h, gap, lbl_w, bar_w = 24, 4, 30, 220
    svg_h = n * (bar_h + gap) + gap
    svg_w = lbl_w + bar_w + 70
    bars = ""
    for i, (etiq, val) in enumerate(zip(etiquettes, valeurs, strict=False)):
        pct = val / max_val if max_val > 0 else 0
        w = max(pct * bar_w, 2)
        y = i * (bar_h + gap) + gap
        delay = f"{i * 0.08:.2f}"
        bars += (
            f'<text x="{lbl_w - 4}" y="{y + bar_h * 0.72}" '
            f'text-anchor="end" font-size="13" font-weight="bold" '
            f'fill="#333">{etiq}</text>'
            f'<rect x="{lbl_w}" y="{y}" width="0" height="{bar_h}" '
            f'rx="4" fill="{couleur}" opacity="0.85">'
            f'<animate attributeName="width" from="0" to="{w:.0f}" '
            f'dur="0.5s" begin="{delay}s" fill="freeze"/></rect>'
            f'<text x="{lbl_w + bar_w + 8}" y="{y + bar_h * 0.72}" '
            f'font-size="12" fill="#555">{val:.0%}</text>'
        )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<svg width="{svg_w}" height="{svg_h}" '
            f'style="margin-top:4px;display:block">{bars}</svg></div>'
        )
    )


def afficher_attention(poids, positions, titre="Poids d'attention"):
    """Attention BertViz-style : lignes entre tokens, opacite = poids."""
    n = len(positions)
    col_w, row_h = 60, 36
    svg_w = col_w * n + 40
    svg_h = row_h * 3 + 20
    elems = ""
    for i, c in enumerate(positions):
        x = 20 + i * col_w + col_w // 2
        elems += (
            f'<text x="{x}" y="20" text-anchor="middle" '
            f'font-size="16" font-weight="bold" fill="#333">{c}</text>'
        )
        w = poids[i]
        opacity = max(w, 0.05)
        stroke_w = 1 + w * 8
        elems += (
            f'<line x1="{x}" y1="28" x2="{svg_w // 2}" y2="{svg_h - 30}" '
            f'stroke="#1565c0" stroke-width="{stroke_w:.1f}" '
            f'stroke-opacity="{opacity:.2f}" stroke-linecap="round"/>'
        )
        elems += (
            f'<text x="{x}" y="42" text-anchor="middle" '
            f'font-size="11" fill="#555">{w:.0%}</text>'
        )
    elems += (
        f'<text x="{svg_w // 2}" y="{svg_h - 10}" text-anchor="middle" '
        f'font-size="14" font-weight="bold" fill="#1565c0">query</text>'
    )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<svg width="{svg_w}" height="{svg_h}" '
            f'style="margin-top:4px;display:block">{elems}</svg></div>'
        )
    )


def afficher_architecture():
    """Affiche le schema de reseau du mini-LLM avec connexions residuelles."""
    # Styles communs
    box = (
        "padding:8px 12px;border-radius:8px;text-align:center;"
        "font-family:system-ui,-apple-system,sans-serif;position:relative;z-index:1"
    )
    dim = "font-size:0.75em;color:#888;font-style:italic"
    arrow = "font-size:1.2em;color:#999"

    html = (
        '<!-- tuto-viz --><div style="margin:8px 0"><b>Architecture du mini-LLM</b>'
        '<div style="display:flex;flex-direction:column;align-items:center;'
        'gap:2px;margin-top:8px;position:relative">'
        # Entree
        f'<div style="background:#f8f9fa;border:2px solid #495057;{box};width:320px">'
        '<b style="color:#495057">\U0001f4e5 Entree</b><br>'
        f'<span style="{dim}">".pik" \u2192 [0, 16, 9, 11]</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
        # Embeddings
        f'<div style="background:#e8f5e9;border:2px solid #2e7d32;{box};width:320px">'
        '<b style="color:#2e7d32">\U0001f3af Token + Position Embedding</b><br>'
        f'<span style="{dim}">tok_emb[id] + pos_emb[pos] \u2192 [n, {"{EMBED_DIM}"}]</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
    )

    # Self-Attention block avec detail Q/K/V + skip connection
    html += (
        '<div style="position:relative;width:380px">'
        # Skip connection gauche (ligne verticale qui contourne l'attention)
        '<div style="position:absolute;left:0;top:0;bottom:0;width:30px">'
        '<div style="border-left:2px dashed #1565c0;border-top:2px dashed #1565c0;'
        "border-bottom:2px dashed #1565c0;height:100%;margin-left:10px;"
        'border-radius:8px 0 0 8px"></div></div>'
        '<div style="position:absolute;left:2px;top:50%;transform:translateY(-50%) rotate(90deg);'
        f'font-size:0.65em;color:#1565c0;white-space:nowrap">\u2795 residuelle</div>'
        # Boite attention
        '<div style="margin-left:35px;margin-right:5px">'
        f'<div style="background:#e3f2fd;border:2px solid #1565c0;{box}">'
        '<b style="color:#1565c0">\U0001f50d Self-Attention</b><br>'
        f'<span style="{dim}">Q = Wq \u00d7 x, K = Wk \u00d7 x, V = Wv \u00d7 x<br>'
        "scores = Q \u00b7 K / \u221ad \u2192 softmax \u2192 \u2211 poids \u00d7 V</span>"
        "</div></div></div>"
        f'<div style="{arrow}">\u25bc + skip</div>'
    )

    # MLP block avec skip connection
    html += (
        '<div style="position:relative;width:380px">'
        # Skip connection gauche (ligne verticale qui contourne le MLP)
        '<div style="position:absolute;left:0;top:0;bottom:0;width:30px">'
        '<div style="border-left:2px dashed #e65100;border-top:2px dashed #e65100;'
        "border-bottom:2px dashed #e65100;height:100%;margin-left:10px;"
        'border-radius:8px 0 0 8px"></div></div>'
        '<div style="position:absolute;left:2px;top:50%;transform:translateY(-50%) rotate(90deg);'
        f'font-size:0.65em;color:#e65100;white-space:nowrap">\u2795 residuelle</div>'
        # Boite MLP
        '<div style="margin-left:35px;margin-right:5px">'
        f'<div style="background:#fff3e0;border:2px solid #e65100;{box}">'
        '<b style="color:#e65100">\U0001f9e0 MLP (ReLU)</b><br>'
        f'<span style="{dim}">W1 \u00d7 x + b1 \u2192 ReLU \u2192 W2 \u00d7 h + b2</span>'
        "</div></div></div>"
        f'<div style="{arrow}">\u25bc + skip</div>'
    )

    # Projection + Softmax + Sortie
    html += (
        f'<div style="background:#fce4ec;border:2px solid #c62828;{box};width:320px">'
        '<b style="color:#c62828">\U0001f4ca Projection + Softmax</b><br>'
        f'<span style="{dim}">W_out \u00d7 x \u2192 logits \u2192 softmax \u2192 probas</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
        f'<div style="background:#f3e5f5;border:2px solid #6a1b9a;{box};width:320px">'
        '<b style="color:#6a1b9a">\U0001f4e4 Sortie</b><br>'
        f'<span style="{dim}">Probabilite pour chaque lettre (27 valeurs)</span></div>'
        "</div>"
        '<div style="margin-top:8px;color:#555;font-size:0.92em;text-align:center">'
        "C'est la meme architecture que GPT-2/3/4, juste en beaucoup plus petit.<br>"
        '<span style="color:#1565c0">Les lignes pointillees</span> sont les '
        "<b>connexions residuelles</b> (raccourcis qui aident l'apprentissage)."
        "</div></div>"
    )
    display(HTML(html))


def afficher_temperature(probas, labels, titre="Effet de la temperature"):
    """Slider temperature interactif avec barres de probas en temps reel."""
    import math as _m

    uid = uuid.uuid4().hex[:8]
    log_p = [_m.log(p + 1e-10) for p in probas]
    data_json = json.dumps({"logp": log_p, "labels": labels})
    n = len(labels)
    bar_h, gap, lbl_w, bar_w = 22, 3, 30, 200
    svg_h = n * (bar_h + gap) + gap + 4
    svg_w = lbl_w + bar_w + 70
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div id="tp{uid}" style="margin:8px 0"><b>{titre}</b>'
            f'<div style="margin-top:6px">'
            f'Temperature : <input type="range" id="sl{uid}" '
            f'min="0.1" max="3.0" step="0.1" value="1.0" '
            f'style="width:200px;vertical-align:middle"> '
            f'<b id="tv{uid}">1.0</b></div>'
            f'<svg id="sv{uid}" width="{svg_w}" height="{svg_h}" '
            f'style="margin-top:4px;display:block"></svg>'
            f'<div style="color:#555;font-size:0.85em;margin-top:4px">'
            f"Deplace le curseur pour voir l'effet de la temperature</div></div>"
            f"<script>(function(){{"
            f"var d={data_json};"
            f'var sl=document.getElementById("sl{uid}");'
            f'var tv=document.getElementById("tv{uid}");'
            f'var svg=document.getElementById("sv{uid}");'
            f'var ns="http://www.w3.org/2000/svg";'
            f"function sm(lp,t){{"
            f"var mx=Math.max.apply(null,lp.map(function(v){{return v/t}}));"
            f"var e=lp.map(function(v){{return Math.exp(v/t-mx)}});"
            f"var s=e.reduce(function(a,b){{return a+b}},0);"
            f"return e.map(function(v){{return v/s}})}}"
            f"function draw(t){{"
            f"while(svg.firstChild)svg.removeChild(svg.firstChild);"
            f"var p=sm(d.logp,t);"
            f"var mx=Math.max.apply(null,p);"
            f"for(var i=0;i<d.labels.length;i++){{"
            f"var y=i*{bar_h + gap}+{gap};"
            f"var w=Math.max(p[i]/mx*{bar_w},2);"
            f'var tl=document.createElementNS(ns,"text");'
            f'tl.setAttribute("x",{lbl_w - 4});tl.setAttribute("y",y+{bar_h}*0.72);'
            f'tl.setAttribute("text-anchor","end");tl.setAttribute("font-size","13");'
            f'tl.setAttribute("font-weight","bold");tl.setAttribute("fill","#333");'
            f"tl.textContent=d.labels[i];svg.appendChild(tl);"
            f'var r=document.createElementNS(ns,"rect");'
            f'r.setAttribute("x",{lbl_w});r.setAttribute("y",y);'
            f'r.setAttribute("width",w);r.setAttribute("height",{bar_h});'
            f'r.setAttribute("rx","4");r.setAttribute("fill","#1565c0");'
            f'r.setAttribute("opacity","0.85");svg.appendChild(r);'
            f'var tv2=document.createElementNS(ns,"text");'
            f'tv2.setAttribute("x",{lbl_w + bar_w + 8});tv2.setAttribute("y",y+{bar_h}*0.72);'
            f'tv2.setAttribute("font-size","12");tv2.setAttribute("fill","#555");'
            f'tv2.textContent=(p[i]*100).toFixed(1)+"%";svg.appendChild(tv2)}}}}'
            f'sl.addEventListener("input",function(){{'
            f"var t=parseFloat(sl.value);tv.textContent=t.toFixed(1);draw(t)}});"
            f"draw(1.0)"
            f"}})();</script>"
        )
    )


def afficher_generation(mot, delai_ms=300):
    """Anime l'apparition d'un mot lettre par lettre."""
    uid = uuid.uuid4().hex[:8]
    lettres_js = json.dumps(list(mot))
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0;font-family:monospace;font-size:1.3em">'
            f'<span id="g{uid}" style="letter-spacing:2px"></span>'
            f'<span id="c{uid}" style="animation:bk{uid} 0.7s infinite">|</span></div>'
            f"<style>@keyframes bk{uid}{{0%,100%{{opacity:1}}50%{{opacity:0}}}}</style>"
            f"<script>(function(){{"
            f"var L={lettres_js},i=0,"
            f'el=document.getElementById("g{uid}"),'
            f'cur=document.getElementById("c{uid}");'
            f"var iv=setInterval(function(){{"
            f'if(i>=L.length){{clearInterval(iv);cur.style.display="none";return}}'
            f'var s=document.createElement("span");'
            f"s.textContent=L[i];"
            f's.style.cssText="opacity:0;transition:opacity 0.3s";'
            f"el.appendChild(s);"
            f'setTimeout(function(){{s.style.opacity="1"}},30);'
            f"i++}},{delai_ms})"
            f"}})();</script>"
        )
    )


print("Outils de visualisation charges !")

---
## Architecture de notre LLM

Notre mini-LLM a la **meme architecture** que GPT-2, GPT-3 et GPT-4.
Seule la taille change. Voyons les pieces :

In [ ]:
import math
import random

random.seed(42)  # pour que tout le monde ait le meme resultat

# --- Vocabulaire (meme que lecon 3) ---
VOCAB = list(".abcdefghijklmnopqrstuvwxyz")
VOCAB_SIZE = len(VOCAB)  # 26 lettres + le point (debut/fin)
char_to_id = {c: i for i, c in enumerate(VOCAB)}
id_to_char = {i: c for i, c in enumerate(VOCAB)}

# --- Configuration du modele ---
# Chaque lettre sera representee par 16 nombres (sa 'fiche d'identite')
EMBED_DIM = 16  # taille des embeddings
CONTEXT = 8  # le modele regarde 8 lettres en arriere (max)
NUM_HEADS = 1  # 1 tete d'attention (pour simplifier)
HEAD_DIM = EMBED_DIM // NUM_HEADS  # = 16
HIDDEN_DIM = 32  # taille du reseau de neurones interne

# Comptons combien de nombres le modele doit apprendre
nb_params = (
    VOCAB_SIZE * EMBED_DIM  # token embeddings       = 432
    + CONTEXT * EMBED_DIM  # position embeddings    = 128
    + 3 * EMBED_DIM * EMBED_DIM  # Q, K, V matrices      = 768
    + EMBED_DIM * HIDDEN_DIM  # MLP couche 1 (poids)  = 512
    + HIDDEN_DIM  # MLP couche 1 (biais)  = 32
    + HIDDEN_DIM * EMBED_DIM  # MLP couche 2 (poids)  = 512
    + EMBED_DIM  # MLP couche 2 (biais)  = 16
    + EMBED_DIM * VOCAB_SIZE  # couche de sortie      = 432
)

print("Configuration du mini-LLM :")
print(f"  Vocabulaire : {VOCAB_SIZE} caracteres")
print(f"  Dimension embeddings : {EMBED_DIM}")
print(f"  Contexte max : {CONTEXT} lettres")
print(f"  Taille MLP : {HIDDEN_DIM}")
print(f"  Nombre de parametres : {nb_params:,}")
print()
print(f"  (GPT-4 en a ~1,800,000,000,000 -- {nb_params / 1.8e12 * 100:.10f}% de GPT-4)")

In [ ]:
# Schema de l'architecture
afficher_architecture()

A toi de changer la taille du modele :

In [ ]:
exercice(
    1,
    "Change la taille du modele",
    "Change <code>EMBED_DIM_test</code> ci-dessous (essaie 8 ou 32), puis <b>Shift + Entree</b>.",
    "Doubler EMBED_DIM quadruple presque le nombre de parametres !",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

EMBED_DIM_test = 16  # <-- Essaie 8 (petit) ou 32 (grand) !

# ╚══════════════════════════════════════╝

HIDDEN_DIM_test = EMBED_DIM_test * 2
nb_params_test = (
    VOCAB_SIZE * EMBED_DIM_test
    + CONTEXT * EMBED_DIM_test
    + 3 * EMBED_DIM_test * EMBED_DIM_test
    + EMBED_DIM_test * HIDDEN_DIM_test
    + HIDDEN_DIM_test
    + HIDDEN_DIM_test * EMBED_DIM_test
    + EMBED_DIM_test
    + EMBED_DIM_test * VOCAB_SIZE
)

print(f"Avec EMBED_DIM = {EMBED_DIM_test} :")
print(f"  Nombre de parametres : {nb_params_test:,}")
print(f"  (Notre modele en a {nb_params:,})")
if EMBED_DIM_test < EMBED_DIM:
    print("  -> Plus petit : moins de parametres, plus rapide, mais moins precis.")
elif EMBED_DIM_test > EMBED_DIM:
    print("  -> Plus grand : plus de parametres, potentiellement meilleur.")

verifier(
    1,
    EMBED_DIM_test != 16,
    f"Bien joue ! Avec {EMBED_DIM_test} dimensions, le modele a {nb_params_test:,} parametres.",
    "Change EMBED_DIM_test pour une autre valeur, par exemple 8 ou 32.",
)

---
## Les briques de base

Avant de construire le LLM, on definit les operations mathematiques
de base. Tu les reconnais des lecons precedentes :

In [ ]:
def rand_matrix(rows, cols, scale=0.3):
    """Cree une matrice aleatoire."""
    return [[random.gauss(0, scale) for _ in range(cols)] for _ in range(rows)]


def rand_vector(size, scale=0.3):
    """Cree un vecteur aleatoire."""
    return [random.gauss(0, scale) for _ in range(size)]


# mat_vec : comme passer a travers un filtre (lecon 3)
def mat_vec(mat, vec):
    """Multiplication matrice x vecteur."""
    return [sum(mat[i][j] * vec[j] for j in range(len(vec))) for i in range(len(mat))]


def vec_add(a, b):
    """Addition de deux vecteurs."""
    return [x + y for x, y in zip(a, b, strict=False)]


# softmax : les scores deviennent des probabilites (lecon 1)
def softmax(scores):
    """Scores -> probabilites (somme = 1)."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]


# relu : porte a sens unique, seules les valeurs positives passent
def relu(x):
    """Si positif, on garde. Si negatif, on met a zero."""
    return [max(0, v) for v in x]


print("Fonctions utilitaires definies !")

---
## Etape 1 : Initialiser les poids

Chaque composant du LLM a ses propres poids (nombres aleatoires au debut).
L'entrainement (lecon 6) trouvera les bonnes valeurs :

In [ ]:
# --- Embeddings (chaque lettre et chaque position ont leur vecteur) ---
tok_emb = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.5)  # lettre -> vecteur
pos_emb = rand_matrix(CONTEXT, EMBED_DIM, 0.5)  # position -> vecteur

# --- Attention : qui est important pour predire la suite ? ---
Wq = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)  # matrice Query
Wk = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)  # matrice Key
Wv = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)  # matrice Value

# --- MLP (2 couches) ---
W1 = rand_matrix(HIDDEN_DIM, EMBED_DIM, 0.2)  # 16 -> 32
b1 = rand_vector(HIDDEN_DIM, 0.1)  # biais
W2 = rand_matrix(EMBED_DIM, HIDDEN_DIM, 0.2)  # 32 -> 16
b2 = rand_vector(EMBED_DIM, 0.1)  # biais

# --- Sortie ---
W_out = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.2)  # 16 -> 27 (une proba par lettre)

print("Modele initialise avec des poids aleatoires.")
print("Il ne sait rien encore -- il faut l'entrainer !")

---
## Etape 2 : Le forward pass (etape par etape)

Le "forward pass" transforme une sequence de lettres en probabilites.
Decomposons chaque etape sur l'exemple **".pik"** :

### Etape 2a : Embeddings (lettre + position)

In [ ]:
# On encode ".pik" : chaque lettre -> vecteur de 16 nombres
test_mot = ".pik"
test_ids = [char_to_id[c] for c in test_mot]  # lettres -> numeros

# Embedding = sens de la lettre + sa position dans le mot
hidden = []
for i, tok_id in enumerate(test_ids):
    # On additionne les 2 embeddings pour avoir une representation complete
    h = vec_add(tok_emb[tok_id], pos_emb[i % CONTEXT])
    hidden.append(h)

print(f"Entree : '{test_mot}' -> ids {test_ids}")
print(f"Apres embeddings : {len(hidden)} vecteurs de {len(hidden[0])} nombres")
print(f"  (chaque lettre est maintenant {EMBED_DIM} nombres)")

### Etape 2b : Self-Attention

L'attention decide **quelles lettres sont importantes** pour predire
la suite (lecon 4). Elle calcule Q, K, V puis les poids :

In [ ]:
# Query = 'que cherche-t-on ?' (pour la derniere position)
q = mat_vec(Wq, hidden[-1])  # 16 nombres

# Key = 'qu'a-t-on a offrir ?' et Value = 'l'info a transmettre'
scores_attn = []
values = []
for i in range(len(test_ids)):
    k = mat_vec(Wk, hidden[i])  # Key de chaque position
    v = mat_vec(Wv, hidden[i])  # Value de chaque position
    # Score = compatibilite entre la question (Q) et la reponse (K)
    score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
    scores_attn.append(score)
    values.append(v)

attn_weights = softmax(scores_attn)  # transformer les scores en poids

print("Poids d'attention (qui est important pour predire apres 'k') :")
for ch, w in zip(test_mot, attn_weights, strict=False):
    print(f"  '{ch}' : {w:.1%}")

# Somme ponderee des Values
attn_out = [0.0] * EMBED_DIM
for i in range(len(test_ids)):
    for d in range(EMBED_DIM):
        attn_out[d] += attn_weights[i] * values[i][d]

# Connexion residuelle (raccourci)
x = vec_add(hidden[-1], attn_out)
print(f"\nApres attention + residuelle : {len(x)} nombres")

# Visualisation BertViz
afficher_attention(attn_weights, list(test_mot), titre="Attention sur '.pik'")

### Etape 2c : MLP + Sortie

Le MLP transforme le resultat de l'attention, puis une derniere
couche convertit en 27 scores (un par lettre) :

In [ ]:
# MLP : le reseau de neurones interne transforme le resultat de l'attention
h = relu(vec_add(mat_vec(W1, x), b1))  # 32 nombres (couche cachee)
mlp_out = vec_add(mat_vec(W2, h), b2)  # 16 nombres (retour a EMBED_DIM)
# Connexion residuelle : garder l'original + ajouter la nouveaute
x = vec_add(x, mlp_out)

# Sortie : les 16 nombres deviennent 27 scores (un par lettre)
logits = mat_vec(W_out, x)  # scores bruts (peuvent etre negatifs)
probas = softmax(logits)  # probabilites (entre 0 et 1, somme = 1)

# Top 5 predictions
top5 = sorted(range(VOCAB_SIZE), key=lambda i: -probas[i])[:5]
print("Predictions (avant entrainement) pour la lettre apres '.pik' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("\n  (C'est du hasard pour l'instant - il faut entrainer !)")

afficher_barres(
    [probas[i] for i in top5],
    [id_to_char[i] for i in top5],
    titre="Top 5 predictions apres '.pik' (avant entrainement)",
)

---
## La fonction complete

On assemble les 3 etapes en une seule fonction `forward_llm` :

In [ ]:
# Cette fonction assemble les 3 etapes en une seule
def forward_llm(sequence_ids):
    """Passe une sequence dans le mini-LLM et retourne les probas."""
    n = len(sequence_ids)

    # 1. Embeddings : sens de la lettre + position dans le mot
    hidden = []
    for i, tok_id in enumerate(sequence_ids):
        h = vec_add(tok_emb[tok_id], pos_emb[i % CONTEXT])
        hidden.append(h)

    # 2. Self-Attention (sur la derniere position)
    q = mat_vec(Wq, hidden[-1])
    scores = []
    values = []
    for i in range(n):
        k = mat_vec(Wk, hidden[i])
        v = mat_vec(Wv, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores.append(score)
        values.append(v)
    attn_weights = softmax(scores)
    attn_out = [0.0] * EMBED_DIM
    for i in range(n):
        for d in range(EMBED_DIM):
            attn_out[d] += attn_weights[i] * values[i][d]
    x = vec_add(hidden[-1], attn_out)  # + residuelle

    # 3. MLP
    h = relu(vec_add(mat_vec(W1, x), b1))
    mlp_out = vec_add(mat_vec(W2, h), b2)
    x = vec_add(x, mlp_out)  # + residuelle

    # 4. Sortie
    logits = mat_vec(W_out, x)
    return softmax(logits)


def _calculer_poids_attention(texte):
    """Calcule les poids d'attention pour un texte."""
    ids = [char_to_id[c] for c in texte]
    hidden = [vec_add(tok_emb[tid], pos_emb[i % CONTEXT]) for i, tid in enumerate(ids)]
    q = mat_vec(Wq, hidden[-1])
    scores = []
    for i in range(len(ids)):
        k = mat_vec(Wk, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores.append(score)
    return softmax(scores)


# Verification rapide
check = forward_llm(test_ids)
print(f"forward_llm('.pik') -> {len(check)} probabilites, somme = {sum(check):.4f}")
print("Fonction complete prete !")

Testons le modele avec differents debuts de mots :

In [ ]:
exercice(
    2,
    "Change le debut du mot",
    'Change <code>mon_debut</code> ci-dessous (essaie <code>".bul"</code>, <code>".evo"</code> ou <code>".dra"</code>).',
    "Le modele predit des lettres differentes selon le debut.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

mon_debut = ".pik"  # <-- Essaie ".bul", ".evo" ou ".dra" !

# ╚══════════════════════════════════════╝

test_ids = [char_to_id[c] for c in mon_debut]
probas = forward_llm(test_ids)
top5 = sorted(range(VOCAB_SIZE), key=lambda i: -probas[i])[:5]

print(f"Predictions apres '{mon_debut}' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("\n(Le modele n'est pas entraine, donc c'est du hasard !)")

afficher_barres(
    [probas[i] for i in top5],
    [id_to_char[i] for i in top5],
    titre=f"Top 5 predictions apres '{mon_debut}'",
)
afficher_attention(
    _calculer_poids_attention(mon_debut),
    list(mon_debut),
    titre=f"Poids d'attention pour '{mon_debut}'",
)

verifier(
    2,
    mon_debut != ".pik",
    f"Super ! Tu as teste les predictions apres '{mon_debut}'.",
    "Change mon_debut pour un autre debut, par exemple '.bul' ou '.evo'.",
)

---
## La loss du modele

Comme dans la lecon 2, on mesure l'erreur du modele avec la **loss**.
Un modele aleatoire a une loss de ~3.3 (= log(27)).
Apres entrainement, elle devrait descendre bien plus bas :

In [ ]:
pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]


def calcul_loss(pokemons):
    """Calcule la loss moyenne sur tous les Pokemon."""
    loss_totale = 0
    nb = 0
    for pokemon in pokemons:
        mot = "." + pokemon + "."
        ids = [char_to_id[c] for c in mot]
        for i in range(1, len(ids)):
            seq = ids[:i]
            cible = ids[i]  # la bonne reponse
            probas = forward_llm(seq[-CONTEXT:])
            # -log(proba) : si le modele est sur -> petite loss, incertain -> grande loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1
    return loss_totale / nb  # moyenne sur toutes les positions


loss_initiale = calcul_loss(pokemons)
print(f"Loss initiale : {loss_initiale:.3f}")
print(f"(Loss d'un modele aleatoire : {math.log(VOCAB_SIZE):.3f})")
print()
print("L'entrainement complet se fait dans la lecon 6.")
print("Mais l'ARCHITECTURE est exactement la meme que GPT-2/3/4 !")

---
## Generation de noms

Meme sans entrainement, on peut voir le **mecanisme** de generation :
le modele genere une lettre a la fois, en se basant sur les probabilites :

In [ ]:
# Generer un nom lettre par lettre, comme ChatGPT genere mot par mot
def generer_llm(debut=".", temperature=1.0, max_len=15):
    """Genere un nom de Pokemon lettre par lettre."""
    ids = [char_to_id[c] for c in debut]
    resultat = debut
    for _ in range(max_len):
        probas = forward_llm(ids[-CONTEXT:])  # predire la prochaine lettre
        # Temperature : basse = toujours la meme chose, haute = surprenant
        if temperature != 1.0:
            logits = [math.log(p + 1e-10) / temperature for p in probas]
            probas = softmax(logits)
        idx = random.choices(range(VOCAB_SIZE), weights=probas, k=1)[0]
        if idx == char_to_id["."]:
            break
        ids.append(idx)
        resultat += id_to_char[idx]
    return resultat[1:] if resultat.startswith(".") else resultat


print("Noms generes (modele non-entraine, juste le mecanisme) :")
print()
noms = [generer_llm(temperature=0.8).capitalize() for _ in range(10)]
for nom in noms:
    print(f"  {nom}")
print()
print("C'est du charabia car le modele n'est pas entraine.")
print("Mais le MECANISME est exactement celui de ChatGPT !")

# Animation du dernier nom genere
afficher_generation(noms[-1], delai_ms=250)

---
## La temperature

La **temperature** controle la creativite du modele :
- **T < 1** : conservateur (toujours les lettres les plus probables)
- **T = 1** : normal
- **T > 1** : creatif (explore des combinaisons inhabituelles)

Deplace le curseur ci-dessous pour voir l'effet en temps reel :

In [ ]:
# Calcul des probas pour le slider interactif
_probas_temp = forward_llm([char_to_id[c] for c in ".pik"])
_top_idx = sorted(range(VOCAB_SIZE), key=lambda i: -_probas_temp[i])[:10]
afficher_temperature(
    [_probas_temp[i] for i in _top_idx],
    [id_to_char[i] for i in _top_idx],
    titre="Effet de la temperature sur les predictions apres '.pik'",
)

A toi de generer avec differentes temperatures :

In [ ]:
exercice(
    3,
    "Joue avec la temperature",
    "Change <code>ma_temperature</code> ci-dessous (essaie 0.1 ou 2.0).",
    "Temperature basse = repetitif. Temperature haute = creatif.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

ma_temperature = 0.8  # <-- Essaie 0.1 (sage) ou 2.0 (fou) !

# ╚══════════════════════════════════════╝

print(f"Generation avec temperature = {ma_temperature} :")
print()
for _ in range(10):
    nom = generer_llm(temperature=ma_temperature).capitalize()
    print(f"  {nom}")
print()
if ma_temperature < 0.5:
    print(
        "Temperature basse : le modele choisit toujours les lettres les plus probables."
    )
elif ma_temperature > 1.5:
    print("Temperature haute : le modele explore des combinaisons inhabituelles !")
else:
    print("Temperature moyenne : un bon equilibre entre creativite et coherence.")

verifier(
    3,
    ma_temperature != 0.8,
    f"Genial ! Tu as explore la temperature {ma_temperature}.",
    "Change ma_temperature pour une autre valeur, par exemple 0.1 ou 2.0.",
)

---
## Ce qu'on a appris

```
Lecon 1 : Compter les lettres qui suivent        -> bigramme
Lecon 2 : Apprendre de ses erreurs                -> entrainement
Lecon 3 : Regarder plusieurs lettres en arriere   -> embeddings + contexte
Lecon 4 : Choisir les lettres importantes          -> attention
Lecon 5 : Assembler le tout                       -> mini-LLM !
```

## La difference avec ChatGPT

| | Notre mini-LLM | ChatGPT |
|---|---|---|
| Architecture | La meme ! | La meme ! |
| Parametres | ~2,800 | ~1,800,000,000,000 |
| Donnees | 20 Pokemon | Internet entier |
| Calcul | 1 PC, secondes | Des milliers de GPU, des mois |
| Resultat | Noms de Pokemon inventes | Conversations, code, poesie... |

L'algorithme est **le meme**. La seule difference, c'est l'echelle.

> *"This file is the complete algorithm. Everything else is just efficiency."*
> -- Andrej Karpathy

---

**Felicitations ! Tu as compris comment fonctionne un LLM.**

*Prochaine lecon : [06 - Entrainer le modele](06_entrainer_le_modele.ipynb)*

---

### Sources (ISO 42001)

- **Architecture complete GPT (embedding + attention + MLP + softmax)** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) \u2014 Andrej Karpathy
- **Comparaison des parametres GPT-4** : estimations publiques basees sur les rapports techniques OpenAI
- **Explication du forward pass complet** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) \u2014 Andrej Karpathy (2023)
- **Concept de temperature pour la generation** : meme source, section sampling
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762)
- **Dataset Pokemon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage educatif. Source : [PokeAPI](https://pokeapi.co/)